# Create OCEAN Data NFT
Quickly create OCEAN Data NFTs by querying blockchain data from Dune and Flipside.

**Steps:**
1. Get Data
2. Store Data
3. Create Data NFT

Alternative Option: Build Dune/Flipside queries on platforms and create OCEAN Data NFTs from UI.

In [1]:
import json
import pandas as pd
import os
from dotenv import load_dotenv
import requests
import subprocess
from flipside import Flipside
from ocean_lib.web3_internal.utils import connect_to_network
from ocean_lib.example_config import get_config_dict
from ocean_lib.ocean.ocean import Ocean
from brownie.network import accounts

# Load variables from .env file
load_dotenv()
DUNE_API_KEY = os.getenv("DUNE_API_KEY")
FLIPSIDE_API_KEY = os.getenv("FLIPSIDE_API_KEY")

## 1. Get Data
### Flipside

* SDK Docs: https://docs.flipsidecrypto.com/flipside-api/get-started

In [2]:
# Query Flipside using their Python SDK
def query_flipside(sql):
    flipside = Flipside(FLIPSIDE_API_KEY, "https://api-v2.flipsidecrypto.xyz")
    results = flipside.query(sql)
    results_df = pd.DataFrame(results.rows, columns=results.columns).drop(columns=["__row_index"])
    return results_df

In [3]:
sql = """
SELECT
  date(block_timestamp) as dt,
  count(distinct tx_hash) as tx_ct
FROM ethereum.core.fact_transactions
WHERE block_timestamp >= GETDATE() - interval'7 days'
GROUP BY 1
order by 1 asc
"""
results_df = query_flipside(sql)
results_df

dt    tx_ct
0  2023-08-07T00:00:00.000Z   431242
1  2023-08-08T00:00:00.000Z  1098689
2  2023-08-09T00:00:00.000Z  1076595
3  2023-08-10T00:00:00.000Z  1037733
4  2023-08-11T00:00:00.000Z  1050203
5  2023-08-12T00:00:00.000Z   972839
6  2023-08-13T00:00:00.000Z   920802
7  2023-08-14T00:00:00.000Z   569926

### Dune

* API Docs: https://dune.com/docs/api/

In [5]:
# Query Dune Analytics via API
def query_dune(q):
    url = f"https://api.dune.com/api/v1/query/{q}/results?api_key={DUNE_API_KEY}"
    response = requests.get(url)
    results_json = json.loads(response.text)["result"]["rows"]
    results_df = pd.DataFrame.from_dict(results_json)
    return results_df

In [6]:
dune_query_id = 2847682
results_df = query_dune(dune_query_id)
results_df

dt    tx_ct
0  2023-08-05  1124372
1  2023-08-06   987506
2  2023-08-07  1047336
3  2023-08-08  1098689
4  2023-08-09  1076595
5  2023-08-10  1037733
6  2023-08-11  1050203
7  2023-08-12    98194

## 2. Store Data
### Add File to GitHub

*Note: This is a quick & easy solution to host the data, but can also add file to IPFS, Arweave, GCP, AWS, etc...*

In [8]:
# Create File from Query results
results_df.to_csv('query_results2.csv', index=False)

In [ ]:
# Store file in GitHub repo
def git_add_commit_push(commit_message):
    try:
        subprocess.run(['git', 'add', '.'])
        subprocess.run(['git', 'commit', '-m', commit_message])
        subprocess.run(['git', 'push'])
        print("Git add, commit, and push successful!")
    except Exception as e:
        print("An error occurred:", e)

# Replace 'Your commit message here' with your desired commit message
commit_message = 'Your commit message here'

git_add_commit_push(commit_message)

## 3. Create Data NFT
### Add Data NFT to the OCEAN Protocol Marketplace

* Marketplace: https://market.oceanprotocol.com/
* SDK Docs: https://docs.oceanprotocol.com/developers/ocean.py/install

In [ ]:
# Create to Blockchain
connect_to_network("polygon-main")
config = get_config_dict("polygon-main")
ocean = Ocean(config)

In [ ]:
# Connect to Wallet
accounts.clear()
private_key = os.getenv('PRIVATE_KEY')
wallet = accounts.add(private_key)
print('MATIC Balance:', wallet.balance() / 1e18 )

In [ ]:
# Create Data NFT
name = "Data NFT Test 1"
url = "https://raw.githubusercontent.com/PrimoData/ocean_data_nfts/main/query_results.csv"
tx_dict = {
    "from": wallet,
    "priority_fee": "500 gwei",
    "max_fee": "500 gwei",
}
(data_nft, datatoken, ddo) = ocean.assets.create_url_asset(name, url, tx_dict)

print("Data NFT published:")
print(f"  data_nft: symbol={data_nft.symbol()}, address={data_nft.address}")
print(f"  datatoken: symbol={datatoken.symbol()}, address={datatoken.address}")
print(f"  did={ddo.did}")